In [1]:
url = "https://seattle.eater.com/maps/best-restaurants-seattle-38"

In [2]:
from citoplasm.tools.search import search_json


search_results = await search_json("Seattle restaurants site:eater.com")

In [3]:
search_results['organic_results'][0]['link']

'https://seattle.eater.com/maps/best-restaurants-seattle-38'

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
response = requests.get(url)


In [6]:
response.status_code

200

In [7]:
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
soup


<!DOCTYPE html>

<html lang="en">
<head>
<title>Best Restaurants in Seattle - Eater Seattle</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="Eater Seattle" name="apple-mobile-web-app-title">
<meta content="https://www.facebook.com/pages/Eater-Seattle/213198508690703" property="article:publisher">
<meta content="Eater Staff" property="author">
<meta content="2018-04-10T12:45:32-07:00" property="article:published_time"/>
<meta content="2023-04-11T12:41:37-07:00" property="article:modified_time"/>
<meta content="6029273333,242726355745404,115169205199800,139755792796580,131547960225574,213601342019363,301223156562915,309491225851948,243335249017579,348494641841991,109814609047278,442143005803582,110073695686645,385346484849141,451388614896528,240433149442075,670887879603309,150611155024023,176453365796072

In [7]:
# get all the h1s
soup.find_all('h1')

[<h1>The 38 Essential Restaurants in Seattle</h1>,
 <h1>Ono Authentic Hawaiian Poke</h1>,
 <h1>Cafe Juanita</h1>,
 <h1>FlintCreek Cattle Co</h1>,
 <h1>Cafe Munir</h1>,
 <h1>Beast and Cleaver</h1>,
 <h1>The Chicken Supply</h1>,
 <h1>Un Bien</h1>,
 <h1>Frelard Tamales</h1>,
 <h1>Pestle Rock</h1>,
 <h1>The Walrus and the Carpenter</h1>,
 <h1>Off The Rez Cafe</h1>,
 <h1>Lupo</h1>,
 <h1>Kamonegi</h1>,
 <h1>Meesha</h1>,
 <h1>Saint Bread</h1>,
 <h1>Joule</h1>,
 <h1>Sushi Kappo Tamura</h1>,
 <h1>Canlis</h1>,
 <h1>Dan Gui Sichuan Cuisine</h1>,
 <h1>Taurus Ox</h1>,
 <h1>Monsoon Seattle</h1>,
 <h1>Spinasse</h1>,
 <h1>Communion Restaurant and Bar</h1>,
 <h1>Sushi Kashiba</h1>,
 <h1>Cafe Campagne</h1>,
 <h1>Oriental Mart</h1>,
 <h1>Nirmal’s</h1>,
 <h1>Pho Bac Sup Shop</h1>,
 <h1>Musang</h1>,
 <h1>Bar del Corso</h1>,
 <h1>Off Alley</h1>,
 <h1>Archipelago</h1>,
 <h1>Delish Ethiopian Cuisine</h1>,
 <h1>Taqueria la Fondita</h1>,
 <h1>Tomo</h1>,
 <h1>El Cabrito</h1>,
 <h1>May Kitchen and Bar</h1>,
 <h1>

In [11]:
# def parse(h1):
#   name = h1.text

#   address_div = h1.find_next_sibling(class_='c-mapstack__address-header')
#   if address_div is None:
#     return None
#   address = address_div.select_one('a').text

#   description = h1.find_next_sibling(class_='c-entry-content').find('p').text

#   return {
#     'name': name,
#     'address': address,
#     'description': description
#   }
from urllib.parse import urlparse, parse_qs

def parse(h1):
  name = h1.text

  address_div = h1.find_next_sibling(class_='c-mapstack__address-header')
  if address_div is None:
    return None
  address = address_div.select_one('a').text

  description = h1.find_next_sibling(class_='c-entry-content').find('p').text

  place_id_link = h1.find_next_sibling('ul', class_='services').find('a')
  if place_id_link is not None:
    url = place_id_link.get('href')
    parsed_url = urlparse(url)
    place_id = parse_qs(parsed_url.query).get('query_place_id', [None])[0]
  else:
    place_id = None

  return {
    'name': name,
    'address': address,
    'description': description,
    'place_id': place_id
  }

restaurants = []
for h1 in soup.find_all('h1'):
  res = parse(h1)
  if res is not None:
    restaurants.append(res)

In [12]:
restaurants

[{'name': 'Ono Authentic Hawaiian Poke',
  'address': '10016 Edmonds Way, Edmonds, WA 98020',
  'description': 'Ono, named for Oahu-born owner Steven Ono, is a seafood lover’s dream in Edmonds, offering possibly the highest-quality poke in the Seattle area on a menu that rotates depending on what’s fresh. Just as importantly, the restaurant uses a light hand with flavorings so you don’t lose the flavor of the fish itself, whether you’re eating salmon dressed with ponzu or a spicy ahi bowl.',
  'place_id': 'ChIJ7fputmIakFQR0OYvJRe0pOM'},
 {'name': 'Cafe Juanita',
  'address': '9702 NE 120th Pl, Kirkland, WA 98034',
  'description': 'Kirkland’s Northern Italian fine dining mainstay offers several excellent seasonal tasting menus, including pescatarian, vegetarian, and vegan options. Chef Holly Smith’s omnivorous tasting menus include dishes like Anderson Valley lamb chops with turnip gratin, while a vegan tasting menu features meat-free marvels like green garlic soup with fava bean toast

In [15]:
from citoplasm.tools.search import search_json
# from ice.utils import map_async

async def search_restaurant(restaurant):
  return await search_json(f"{restaurant['name']} {restaurant['address']}")

# restaurant_search_results = await map_async(restaurants, search_restaurant)

restaurant_data = []
for restaurant in restaurants:
  try:
    search_results = await search_restaurant(restaurant)
    knowledge_graph = search_results['knowledge_graph']
    restaurant_data.append({
    'name': knowledge_graph['title'] if 'title' in knowledge_graph else None,
    'type': knowledge_graph['type'] if 'type' in knowledge_graph else None,
    'website': knowledge_graph['website'] if 'website' in knowledge_graph else None,
    'rating': knowledge_graph['rating'] if 'rating' in knowledge_graph else None,
    'review_count': knowledge_graph['review_count'] if 'review_count' in knowledge_graph else None,
    'price': knowledge_graph['price'] if 'price' in knowledge_graph else None,
    'hours': knowledge_graph['hours'] if 'hours' in knowledge_graph else None,
    'user_reviews': knowledge_graph['user_reviews'] if 'user_reviews' in knowledge_graph else None,
  })
  except Exception as e:
    print(f"Failed to search {restaurant['name']}", e)
    pass


In [16]:
restaurant_data

[{'name': 'Ono Authentic Hawaiian Poke',
  'type': 'Hawaiian restaurant',
  'website': 'http://eatonopoke.com/',
  'rating': 4.8,
  'review_count': 1018,
  'price': '$$',
  'hours': {'monday': {'opens': 'Closed', 'closes': 'Closed'},
   'tuesday': {'opens': '11\u202fAM', 'closes': '7\u202fPM'},
   'wednesday': {'opens': '11\u202fAM', 'closes': '7\u202fPM'},
   'thursday': {'opens': '11\u202fAM', 'closes': '7\u202fPM'},
   'friday': {'opens': '11\u202fAM', 'closes': '7\u202fPM'},
   'saturday': {'opens': '11\u202fAM', 'closes': '7\u202fPM'},
   'sunday': {'opens': '11\u202fAM', 'closes': '5\u202fPM'}},
  'user_reviews': [{'summary': 'Atmosphere is nice, staff friendly, but variety and price not so much.',
    'link': 'https://www.google.com/maps/reviews/data=!4m5!14m4!1m3!1m2!1s115111099529892253632!2s0x54901a62b66efaed:0xe3a4b417252fe6d0?sa=X&ved=2ahUKEwiSypqwy-b-AhUzrVYBHWzXA-MQv_QBegQIdBAY',
    'user': {'name': 'Jennifer Zadow',
     'link': 'https://www.google.com/maps/contrib/1151

In [17]:
search_results

{'search_metadata': {'id': '64596025aa6c9509e040feb7',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/e4816247489323ff/64596025aa6c9509e040feb7.json',
  'created_at': '2023-05-08 20:48:37 UTC',
  'processed_at': '2023-05-08 20:48:37 UTC',
  'google_url': 'https://www.google.com/search?q=Antigua+Guatemala+Restaurant+120+Washington+Ave+N%2C+Kent%2C+WA+98032&oq=Antigua+Guatemala+Restaurant+120+Washington+Ave+N%2C+Kent%2C+WA+98032&hl=en&gl=us&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/e4816247489323ff/64596025aa6c9509e040feb7.html',
  'total_time_taken': 3.76},
 'search_parameters': {'engine': 'google',
  'q': 'Antigua Guatemala Restaurant 120 Washington Ave N, Kent, WA 98032',
  'google_domain': 'google.com',
  'hl': 'en',
  'gl': 'us',
  'device': 'desktop'},
 'search_information': {'organic_results_state': 'Results for exact spelling',
  'query_displayed': 'Antigua Guatemala Restaurant 120 Washington Ave N, Kent, WA 98032',
  'to

In [9]:
knowledge_graph = search_results['knowledge_graph']

relevant_knowledge_graph_info = {
  'name': knowledge_graph['title'],
  'type': knowledge_graph['type'],
  'website': knowledge_graph['website'],
  'rating': knowledge_graph['rating'],
  'review_count': knowledge_graph['review_count'],
  'price': knowledge_graph['price'],
  'hours': knowledge_graph['hours'],
  'user_reviews': knowledge_graph['user_reviews'],
}
relevant_knowledge_graph_info

{'name': 'Ono Authentic Hawaiian Poke',
 'type': 'Hawaiian restaurant',
 'website': 'http://eatonopoke.com/',
 'rating': 4.8,
 'review_count': 1018,
 'price': '$$',
 'hours': {'monday': {'opens': 'Closed', 'closes': 'Closed'},
  'tuesday': {'opens': '11\u202fAM', 'closes': '7\u202fPM'},
  'wednesday': {'opens': '11\u202fAM', 'closes': '7\u202fPM'},
  'thursday': {'opens': '11\u202fAM', 'closes': '7\u202fPM'},
  'friday': {'opens': '11\u202fAM', 'closes': '7\u202fPM'},
  'saturday': {'opens': '11\u202fAM', 'closes': '7\u202fPM'},
  'sunday': {'opens': '11\u202fAM', 'closes': '5\u202fPM'}},
 'user_reviews': [{'summary': 'Atmosphere is nice, staff friendly, but variety and price not so much.',
   'link': 'https://www.google.com/maps/reviews/data=!4m5!14m4!1m3!1m2!1s115111099529892253632!2s0x54901a62b66efaed:0xe3a4b417252fe6d0?sa=X&ved=2ahUKEwjE-MSKyOb-AhU3KEQIHa3_AxwQv_QBegQIbRAY',
   'user': {'name': 'Jennifer Zadow',
    'link': 'https://www.google.com/maps/contrib/115111099529892253632

In [12]:
# use Eater to find a list of restaurants and a list of bars
# for each entry, google the place to get its location and hours

In [13]:
import json

json_ld_script = soup.find("script", type="application/ld+json")
    
if json_ld_script:
    restaurants = json.loads(json_ld_script.string)['itemListElement']
    for restaurant in restaurants:
        print(restaurant['item']['name'])

Ono Authentic Hawaiian Poke
Cafe Juanita
FlintCreek Cattle Co
Cafe Munir
Beast and Cleaver
The Chicken Supply
Un Bien
Frelard Tamales
Pestle Rock
The Walrus and the Carpenter
Off The Rez Cafe
Lupo
Kamonegi
Meesha
Saint Bread
Joule
Sushi Kappo Tamura
Canlis
Dan Gui Sichuan Cuisine
Taurus Ox
Monsoon Seattle
Spinasse
Communion Restaurant and Bar
Sushi Kashiba
Cafe Campagne
Oriental Mart
Nirmal’s
Pho Bac Sup Shop
Musang
Bar del Corso
Off Alley
Archipelago
Delish Ethiopian Cuisine
Taqueria la Fondita
Tomo
El Cabrito
May Kitchen and Bar
Antigua Guatemala Restaurant


In [14]:
print(soup.find('div', attrs={'class': 'c-mapstack__card'}))

None


In [15]:
# The provided HTML snippet
html_snippet = '''
<h1>Ono Authentic Hawaiian Poke</h1>
<div class="c-mapstack__address-header">
    <a href="https://www.google.com/maps/search/?api=1&amp;query=Ono+Authentic+Hawaiian+Poke&amp;query_place_id=ChIJ7fputmIakFQR0OYvJRe0pOM" target="_blank">10016 Edmonds Way, Edmonds, WA 98020</a>
</div>
<div class="c-entry-content venu-card">
    <p>Ono, named for Oahu-born owner Steven Ono, is a seafood lover’s dream in Edmonds, offering possibly the highest-quality poke in the Seattle area on a menu that rotates depending on what’s fresh. Just as importantly, the restaurant uses a light hand with flavorings so you don’t lose the flavor of the fish itself, whether you’re eating salmon dressed with ponzu or a spicy ahi bowl.</p>
</div>
'''

# Parse the HTML snippet using Beautiful Soup
soup = BeautifulSoup(html_snippet, 'html.parser')

# Extract the name, location, and description
name = soup.h1.text
location = soup.select_one('.c-mapstack__address-header a').text
description = soup.select_one('.c-entry-content.venu-card p').text

# Print the extracted information
print(f"Name: {name}")
print(f"Location: {location}")
print(f"Description: {description}")

Name: Ono Authentic Hawaiian Poke
Location: 10016 Edmonds Way, Edmonds, WA 98020
Description: Ono, named for Oahu-born owner Steven Ono, is a seafood lover’s dream in Edmonds, offering possibly the highest-quality poke in the Seattle area on a menu that rotates depending on what’s fresh. Just as importantly, the restaurant uses a light hand with flavorings so you don’t lose the flavor of the fish itself, whether you’re eating salmon dressed with ponzu or a spicy ahi bowl.


None


In [15]:
import requests
import json
import os

def get_place_details(place_id, api_key):
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

api_key = os.environ.get('GOOGLE_MAPS_API_KEY')
print(api_key)
place_id = 'ChIJ7fputmIakFQR0OYvJRe0pOM'
place_details = get_place_details(place_id, api_key)

if place_details is not None:
    print(json.dumps(place_details, indent=4))
else:
    print("Failed to get place details.")


None
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "status": "REQUEST_DENIED"
}
